In [3]:
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from datetime import datetime
from talib import abstract
from io import StringIO
import pymysql.cursors
import pandas as pd
import requests
import json
import sys
import os
import re


twse_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-TW,en-US;q=0.8,en;q=0.5,zh;q=0.3',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.twse.com.tw',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Referer': 'https://www.twse.com.tw/',
    'Upgrade-Insecure-Requests': '1' }

tpex_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-TW,en-US;q=0.8,en;q=0.5,zh;q=0.3',
    'Referer': 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430.php?l=zh-tw',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0' }


def get_tradedaytoday():
    base_time = 190000  # 下午7點190000
    today_time = todaytime
    today_date = today_time.date()
    closed_days = []

    print('正在連線證交所取得交易日期清單。')
    try:
        resp = requests.get('https://www.twse.com.tw/holidaySchedule/holidaySchedule?response=csv', headers=tpex_headers)
        resp.encoding = 'big5'
    except Exception as e:
        print('程式結束。錯誤訊息：%s。因為連線交易日期網址的主機無法回應，連線嘗試失敗。' % e)
        sys.exit(1)
    holidays_csv = resp.text.replace('"', '')
    holidays_readlines = holidays_csv.split('\n')
    for holiday_line in holidays_readlines:
        holiday_column = holiday_line.split(',')
        if (len(holiday_column) > 1) and (holiday_column[4] != 'o'):
            holiday_value = holiday_column[
                1]  # [1] 為[日期]  = ['名稱', '日期', '星期', '說明', '備註(* : 市場無交易，僅辦理結算交割...。o : 交易日。)']
            if re.match(r'\d+月\d+日', holiday_value):
                each_holiday_lines = re.findall(r'\d+月\d+日', holiday_value)
                for each_holiday in each_holiday_lines:
                    _mon = each_holiday[:each_holiday.find('月')]
                    _day = each_holiday[each_holiday.find('月') + 1:each_holiday.find('日')]
                    closed_days.append(datetime(today_date.year, int(_mon), int(_day)).date())
    nTimehms = int(today_time.strftime("%H%M%S"))
    day_offset = 0
    if nTimehms < base_time:
        day_offset = 1
    trading_day = today_date - relativedelta(days=day_offset)
    while (trading_day in closed_days) or (datetime.weekday(trading_day) > 4):
        day_offset += 1
        trading_day = today_date - relativedelta(days=day_offset)
    print('今天目前為止最後一個有收盤資料的交易日為: %s' % trading_day)
    return trading_day
todaytime = datetime.today()
tradedaytoday = get_tradedaytoday()

正在連線證交所取得交易日期清單。
今天目前為止最後一個有收盤資料的交易日為: 2023-09-26


In [4]:
# 上市股票每日收盤行情
def get_tseallbut0999(url, accquiredfile):
    try:
        df = pd.read_html(StringIO(requests.get(url, headers=twse_headers).text), encoding='utf-8')[8]
    except Exception as e:
        print('從網頁讀取 %s 資料時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)
    try:
        df.columns = df.columns.get_level_values(2)
        df.drop(df.columns[2:].to_list(), axis=1, inplace=True)
        df.rename(columns={'證券代號': 'symbol_id', '證券名稱': 'abb_zhname'}, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('在處理網頁 %s 資料內容時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))

    return df.copy()

In [5]:
# 上市股票每日收盤行情
url, progress = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date=%s&type=ALLBUT0999' % tradedaytoday.strftime('%Y%m%d'), '上市股票每日收盤行情'
base_df = get_tseallbut0999(url, progress)
base_df

,abb_zhname
symbol_id,
0050,元大台灣50
0051,元大中型100
0052,富邦科技
0053,元大電子
0055,元大MSCI金融
...,...
9944,新麗
9945,潤泰新
9946,三發地產


In [6]:
# 上櫃股票每日收盤行情
def get_otcstkquote(url, accquiredfile):
    try:
        df = pd.read_html(StringIO(requests.get(url, headers=tpex_headers).text), encoding='utf-8')[0]
    except Exception as e:
        print('從網頁讀取 %s 資料時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)
    try:
        df.columns = df.columns.get_level_values(1)
        df.drop(df.columns[2:].to_list(), axis=1, inplace=True)
        df.rename(columns={'代號': 'symbol_id', '名稱': 'abb_zhname'}, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('在處理網頁 %s 資料內容時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)
    return df[:-1].copy()

In [7]:
# 上櫃股票每日收盤行情
url, progress = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=htm&d=%s&se=EW&s=0,asc,0&_=%s' % (str(tradedaytoday.year-1911)+tradedaytoday.strftime('/%m/%d'), str(datetime.now().timestamp()*1000)[:13]), '上櫃股票每日收盤行情'
temp_df = get_otcstkquote(url, progress)
temp_df

,abb_zhname
symbol_id,
006201,元大富櫃50
00679B,元大美債20年
00687B,國泰20年美債
00694B,富邦美債1-3
00695B,富邦美債7-10
...,...
9949,琉園
9950,萬國通
9951,皇田


In [8]:
base_df = pd.concat([base_df, temp_df])
base_df

,abb_zhname
symbol_id,
0050,元大台灣50
0051,元大中型100
0052,富邦科技
0053,元大電子
0055,元大MSCI金融
...,...
9949,琉園
9950,萬國通
9951,皇田


In [9]:
# 上市公司基本資料
def get_t187ap03L(url, accquiredfile):
    try:
        resp = requests.get(url, headers=twse_headers)
        resp.encoding = 'utf-8'
        df = pd.read_csv(StringIO(resp.text), encoding='utf-8', usecols=[1,2,3,4,5,7,14,15,16,27,31], parse_dates=['成立日期', '上市日期'], dtype={'公司代號': str, '營利事業統一編號': str})
    except Exception as e:
        print('從網頁讀取 %s 資料時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)
    try:
        df.rename(columns={
            '公司代號': 'symbol_id',
            '公司名稱': 'name',
            '公司簡稱': 'abb_zhname',
            '外國企業註冊地國': 'registration',
            '產業別': 'industry',
            '營利事業統一編號': 'tax_ID',
            '成立日期': 'found',
            '上市日期': 'listing',
            '普通股每股面額': 'parvalue',
            '英文簡稱': 'abb_enname',
            '網址': 'website'}, inplace=True)
        df['registration'] = df['registration'].replace('－ ', '')
        df['registration'] = df['registration'].replace(' ', '')
        df['website'] = df['website'].apply(lambda url: url if urlparse(url).scheme else 'https://%s' % url)
        parvalue_map = {
            '新台幣                 10.0000元': ['TWD', 10],
            '新台幣                  1.0000元': ['TWD', 1],
            '新台幣                  2.5000元': ['TWD', 2.5],
            '新台幣                  5.0000元': ['TWD', 5],
            '美元                  0.0010元': ['USD', 1],
            '不適用': ['', 0] ,
            '美金0.0050元': ['USD', 0.005],
            '美金0.05元': ['USD', 0.05],
            '泰銖1元': ['THB', 1],
            '泰銖7.5元': ['THB', 7.5],
            '港幣0.1元': ['HKD', 1],
            '0.01': ['', 0.01],
            '-': ['', 0],
            '泰銖10元': ['THB', 10],
            'HKD 0.1': ['HKD', 0.1],
            '0.1': ['', 0.1],
            '港幣0.01元': ['HKD', 0.01] }
        df = df.join(pd.DataFrame(df['parvalue'].apply(lambda currency_map: parvalue_map[currency_map]).tolist(),
            columns = ['currency','par_value']))
        df.drop(['parvalue', 'abb_zhname'], axis=1, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('整理 %s 資料的 map, join 時發生錯誤: %s' % (accquiredfile, e))
        sys.exit(1)
    return df.copy()


In [10]:
# 上市公司基本資料
url, progress = 'http://mopsfin.twse.com.tw/opendata/t187ap03_L.csv', '上市公司基本資料'
self_df = get_t187ap03L(url, progress)
self_df

,name,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value
symbol_id,,,,,,,,,,
1101,臺灣水泥股份有限公司,,1,11913502,1950-12-29,1962-02-09,TCC,http://www.taiwancement.com,TWD,10.0
1102,亞洲水泥股份有限公司,,1,03244509,1957-03-21,1962-06-08,ACC,https://www.acc.com.tw,TWD,10.0
1103,嘉新水泥股份有限公司,,1,11892801,1954-12-13,1969-11-14,CHC,https://www.chcgroup.com.tw,TWD,10.0
1104,環球水泥股份有限公司,,1,07568009,1960-03-21,1971-02-01,UCC,https://www.ucctw.com,TWD,10.0
1108,幸福水泥股份有限公司,,1,40601248,1974-05-07,1990-06-06,LUCKY CEMENT,https://www.luckygrp.com.tw,TWD,10.0
...,...,...,...,...,...,...,...,...,...,...
9944,新麗企業股份有限公司,,20,33375909,1962-09-14,2002-08-26,Shinih,https://www.shinih.com.tw,TWD,10.0
9945,潤泰創新國際股份有限公司,,20,12139612,1977-09-12,1992-04-30,RUENTEX DEVELOP,https://www.rt-develop.com.tw,TWD,10.0
9946,三發地產股份有限公司(原:金革),,14,84466749,1993-09-20,2013-09-17,SANFAR,https://www.sanfar.com.tw,TWD,10.0


In [11]:
# 上櫃公司基本資料
def get_t187ap03O(url, accquiredfile):
    try:
        resp = requests.get(url, headers=twse_headers)
        resp.encoding = 'utf-8'
        df = pd.read_csv(StringIO(resp.text), encoding='utf-8', usecols=[1,2,3,4,5,7,14,15,16,27,31], parse_dates=['成立日期', '上市日期'], dtype={'公司代號': str, '營利事業統一編號': str})
    except Exception as e:
        print('從網頁讀取 %s 資料時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)
    try:
        df.rename(columns={
            '公司代號': 'symbol_id',
            '公司名稱': 'name',
            '公司簡稱': 'abb_zhname',
            '外國企業註冊地國': 'registration',
            '產業別': 'industry',
            '營利事業統一編號': 'tax_ID',
            '成立日期': 'found',
            '上市日期': 'listing',
            '普通股每股面額': 'parvalue',
            '英文簡稱': 'abb_enname',
            '網址': 'website'}, inplace=True)
        df['registration'] = df['registration'].replace('－ ', '')
        df['registration'] = df['registration'].replace(' ', '')
        df['website'] = df['website'].apply(lambda url: url if urlparse(url).scheme else 'https://%s' % url)
        parvalue_map = {
            '新台幣                  0.4000元': ['TWD', 0.4],
            '新台幣                  1.0000元': ['TWD', 1],
            '新台幣                  2.5000元': ['TWD', 2.5],
            '新台幣                  5.0000元': ['TWD', 5],
            '新台幣                 10.0000元': ['TWD', 10],
            '美元                  0.0010元': ['USD', 1],
            '不適用': ['', 0] ,
            '美金0.0050元': ['USD', 0.005],
            '泰銖1元': ['THB', 1],
            '港幣0.1元': ['HKD', 1],
            '0.01': ['', 0.01],
            '-': ['', 0],
            '泰銖10元': ['THB', 10],
            'HKD 0.1': ['HKD', 0.1],
            '0.1': ['', 0.1],
            '港幣0.01元': ['HKD', 0.01] }
        df = df.join(pd.DataFrame(df['parvalue'].apply(lambda currency_map: parvalue_map[currency_map]).tolist(),
            columns = ['currency','par_value']))
        df.drop(['parvalue'], axis=1, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('整理 %s 資料時發生錯誤: %s' % (accquiredfile, e))
        sys.exit(1)
    return df.copy()

In [12]:
# 上櫃公司基本資料
url, progress = 'http://mopsfin.twse.com.tw/opendata/t187ap03_O.csv', '上櫃公司基本資料'
temp_df = get_t187ap03O(url, progress)
temp_df

,name,abb_zhname,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value
symbol_id,,,,,,,,,,,
1240,茂生農經股份有限公司,茂生農經,,33,18795706,1967-02-18,2018-08-08,MORNSUN,https://www.morn-sun.com.tw/,TWD,10.0
1259,安心食品服務股份有限公司,安心,,16,23928945,1990-11-23,2011-12-15,AN-SHIN,http://www.mos.com.tw,TWD,10.0
1264,德麥食品股份有限公司,德麥,,2,23420497,1989-06-29,2015-04-09,TEHMAG,http://www.tehmag.com.tw,TWD,10.0
1268,漢來美食股份有限公司,漢來美食,,16,80352343,2003-01-09,2017-09-27,Hi-Lai Foods,https://www.hilai-foods.com,TWD,10.0
1336,台翰精密科技股份有限公司,台翰,,28,22589593,1987-08-28,2007-11-02,TAIHAN,https://www.thpt.com.tw,TWD,10.0
...,...,...,...,...,...,...,...,...,...,...,...
9949,琉園股份有限公司,琉園,,32,84767175,1994-04-07,2003-11-21,TITTOT,https://www.tittot.com,TWD,10.0
9950,萬國通路股份有限公司,萬國通,,3,73644351,1979-04-25,2004-02-17,EMINENT,https://www.eminent.com,TWD,10.0
9951,皇田工業股份有限公司,皇田,,5,73707377,1983-07-25,2003-12-18,MACAUTO,https://www.macauto-group.com/zh-hant/,TWD,10.0


In [13]:
# 合併上市公司基本資料與上櫃公司基本資料
self_df = pd.concat([self_df, temp_df])
self_df

,name,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value,abb_zhname
symbol_id,,,,,,,,,,,
1101,臺灣水泥股份有限公司,,1,11913502,1950-12-29,1962-02-09,TCC,http://www.taiwancement.com,TWD,10.0,NaN
1102,亞洲水泥股份有限公司,,1,03244509,1957-03-21,1962-06-08,ACC,https://www.acc.com.tw,TWD,10.0,NaN
1103,嘉新水泥股份有限公司,,1,11892801,1954-12-13,1969-11-14,CHC,https://www.chcgroup.com.tw,TWD,10.0,NaN
1104,環球水泥股份有限公司,,1,07568009,1960-03-21,1971-02-01,UCC,https://www.ucctw.com,TWD,10.0,NaN
1108,幸福水泥股份有限公司,,1,40601248,1974-05-07,1990-06-06,LUCKY CEMENT,https://www.luckygrp.com.tw,TWD,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9949,琉園股份有限公司,,32,84767175,1994-04-07,2003-11-21,TITTOT,https://www.tittot.com,TWD,10.0,琉園
9950,萬國通路股份有限公司,,3,73644351,1979-04-25,2004-02-17,EMINENT,https://www.eminent.com,TWD,10.0,萬國通
9951,皇田工業股份有限公司,,5,73707377,1983-07-25,2003-12-18,MACAUTO,https://www.macauto-group.com/zh-hant/,TWD,10.0,皇田


In [14]:
# 上市股票型基金ETF基本資料彙總表
def get_t187ap47_L(url, accquiredfile):
    try:
        resp = requests.get(url, headers=twse_headers)
        resp.encoding = 'utf-8'
        '''
        出表日期0,基金代號1,基金名稱2,標的指數/追蹤指數名稱3,總機4,基金類型5,英文名稱6,地址7,董事長8,發言人9,總經理10,代理發言人11,投資經理人12,統一編號13,總代理人14,上市日期15,成立日期16,發行單位數/轉換數17,過戶機構18,過戶電話19,過戶地址20,指數股票型基金(ETF)之標的指數是否包含外國成份股21,備註22
        '''
        df = pd.read_csv(StringIO(resp.text), encoding='utf-8', usecols=[1, 2, 3, 5, 6, 13, 15, 16],parse_dates=['上市日期', '成立日期'], dtype={'基金代號': str, '統一編號': str}, date_parser=lambda x: datetime.strptime('%04d-%02d-%02d' % (int(x[0:3]) + 1911, int(x[3:5]), int(x[5:])), '%Y-%m-%d').date())
    except Exception as e:
        print('從網頁讀取 %s 資料時發生錯誤。錯誤訊息: %s' % (accquiredfile, e))
        sys.exit(1)

    df['基金代號'] = df['基金代號'].str.replace('"', '')
    df['基金代號'] = df['基金代號'].str.replace('=', '')
    try:
        df.rename(columns={
            '基金代號': 'symbol_id',
            '基金名稱': 'name',
            '標的指數/追蹤指數名稱': 'description',
            '基金類型': 'type',
            '英文名稱': 'enname',
            '統一編號': 'tax_ID',
            '上市日期': 'listing',
            '成立日期': 'found'}, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('整理 %s 資料時發生錯誤: %s' % (accquiredfile, e))
        sys.exit(1)
    return df.copy()


In [15]:
# 上市 ETF 基金基本資料彙總表上市股票型基金ETF基本資料彙總表
url, progress = 'https://mopsfin.twse.com.tw/opendata/t187ap47_L.csv', '上市 ETF 基金基本資料彙總表'
temp_df = get_t187ap47_L(url, progress)
temp_df

C:\Users\alvinlin\AppData\Local\Temp\ipykernel_6652\1438563364.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(StringIO(resp.text), encoding='utf-8', usecols=[1, 2, 3, 5, 6, 13, 15, 16],parse_dates=['上市日期', '成立日期'], dtype={'基金代號': str, '統一編號': str}, date_parser=lambda x: datetime.strptime('%04d-%02d-%02d' % (int(x[0:3]) + 1911, int(x[3:5]), int(x[5:])), '%Y-%m-%d').date())


,name,description,type,enname,tax_ID,listing,found
symbol_id,,,,,,,
0050,元大台灣卓越50證券投資信託基金,臺灣50指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Top 50 ETF,98764389,2003-06-30,2003-06-25
0051,元大台灣中型100證券投資信託基金,臺灣中型100指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Mid-Cap 100 ETF,48772770,2006-08-31,2006-08-24
0052,富邦台灣科技指數證券投資信託基金,臺灣資訊科技指數,國內成分證券指數股票型基金,Fubon Taiwan Technology Tracker Fund,48790067,2006-09-12,2006-08-28
0053,元大台灣ETF傘型證券投資信託基金之電子科技證券投資信託基金,電子類加權股價指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Electronics Tech ETF,48903196,2007-07-16,2007-07-04
0054,元大台灣ETF傘型證券投資信託基金之台商收成證券投資信託基金,S&amp;P台商收成指數,國內成分證券指數股票型基金,Yuanta/P-shares S&amp;P Custom China Play 50 ETF,48903180,2007-07-16,2007-07-04
...,...,...,...,...,...,...,...
00926,凱基全球菁英55 ETF證券投資信託基金,彭博全球菁英 55 指數,國外成分證券指數股票型基金,KGI Global Industry Elite 55 ETF,88396230,2023-06-02,2023-05-26
00927,群益台灣半導體收益ETF證券投資信託基金,臺灣指數公司特選臺灣上市上櫃半導體收益指數,國內成分證券指數股票型基金,CAPITAL TIP CUSTOMIZED TAIWAN SEMICONDUCTOR DI...,88397435,2023-06-06,2023-05-29
00929,復華台灣科技優息ETF證券投資信託基金,臺灣指數公司特選臺灣上市上櫃科技優息指數,國內成分證券指數股票型基金,Fuh Hwa Taiwan Technology Dividend Highlight ETF,88399314,2023-06-09,2023-06-01


In [16]:
# 合併上市上櫃公司基本資料與上市ETF基金基本資料
self_df = pd.concat([self_df, temp_df])
self_df

,name,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value,abb_zhname,description,type,enname
symbol_id,,,,,,,,,,,,,,
1101,臺灣水泥股份有限公司,,1.0,11913502,1950-12-29,1962-02-09,TCC,http://www.taiwancement.com,TWD,10.0,NaN,NaN,NaN,NaN
1102,亞洲水泥股份有限公司,,1.0,03244509,1957-03-21,1962-06-08,ACC,https://www.acc.com.tw,TWD,10.0,NaN,NaN,NaN,NaN
1103,嘉新水泥股份有限公司,,1.0,11892801,1954-12-13,1969-11-14,CHC,https://www.chcgroup.com.tw,TWD,10.0,NaN,NaN,NaN,NaN
1104,環球水泥股份有限公司,,1.0,07568009,1960-03-21,1971-02-01,UCC,https://www.ucctw.com,TWD,10.0,NaN,NaN,NaN,NaN
1108,幸福水泥股份有限公司,,1.0,40601248,1974-05-07,1990-06-06,LUCKY CEMENT,https://www.luckygrp.com.tw,TWD,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00926,凱基全球菁英55 ETF證券投資信託基金,NaN,NaN,88396230,2023-05-26,2023-06-02,NaN,NaN,NaN,NaN,NaN,彭博全球菁英 55 指數,國外成分證券指數股票型基金,KGI Global Industry Elite 55 ETF
00927,群益台灣半導體收益ETF證券投資信託基金,NaN,NaN,88397435,2023-05-29,2023-06-06,NaN,NaN,NaN,NaN,NaN,臺灣指數公司特選臺灣上市上櫃半導體收益指數,國內成分證券指數股票型基金,CAPITAL TIP CUSTOMIZED TAIWAN SEMICONDUCTOR DI...
00929,復華台灣科技優息ETF證券投資信託基金,NaN,NaN,88399314,2023-06-01,2023-06-09,NaN,NaN,NaN,NaN,NaN,臺灣指數公司特選臺灣上市上櫃科技優息指數,國內成分證券指數股票型基金,Fuh Hwa Taiwan Technology Dividend Highlight ETF


In [17]:
# 丟棄中文簡稱名
self_df.drop(['abb_zhname'], axis=1, inplace=True)
# 合併兩組資料
base_df = base_df.join(self_df)
base_df

,abb_zhname,name,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value,description,type,enname
symbol_id,,,,,,,,,,,,,,
0050,元大台灣50,元大台灣卓越50證券投資信託基金,NaN,NaN,98764389,2003-06-25,2003-06-30,NaN,NaN,NaN,NaN,臺灣50指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Top 50 ETF
0051,元大中型100,元大台灣中型100證券投資信託基金,NaN,NaN,48772770,2006-08-24,2006-08-31,NaN,NaN,NaN,NaN,臺灣中型100指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Mid-Cap 100 ETF
0052,富邦科技,富邦台灣科技指數證券投資信託基金,NaN,NaN,48790067,2006-08-28,2006-09-12,NaN,NaN,NaN,NaN,臺灣資訊科技指數,國內成分證券指數股票型基金,Fubon Taiwan Technology Tracker Fund
0053,元大電子,元大台灣ETF傘型證券投資信託基金之電子科技證券投資信託基金,NaN,NaN,48903196,2007-07-04,2007-07-16,NaN,NaN,NaN,NaN,電子類加權股價指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Electronics Tech ETF
0055,元大MSCI金融,元大台灣ETF傘型證券投資信託基金之金融證券投資信託基金,NaN,NaN,48903175,2007-07-04,2007-07-16,NaN,NaN,NaN,NaN,MSCI台灣金融指數,國內成分證券指數股票型基金,Yuanta/P-shares MSCI Taiwan Financials ETF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9949,琉園,琉園股份有限公司,,32.0,84767175,1994-04-07,2003-11-21,TITTOT,https://www.tittot.com,TWD,10.0,NaN,NaN,NaN
9950,萬國通,萬國通路股份有限公司,,3.0,73644351,1979-04-25,2004-02-17,EMINENT,https://www.eminent.com,TWD,10.0,NaN,NaN,NaN
9951,皇田,皇田工業股份有限公司,,5.0,73707377,1983-07-25,2003-12-18,MACAUTO,https://www.macauto-group.com/zh-hant/,TWD,10.0,NaN,NaN,NaN


In [18]:
# 上市上櫃國際證券辨識號碼表
def get_isincode(url, accquiredfile):
    composedata = []
    try:
        for strMode in ({ '2':'上市'}, {'4':'上櫃'}):
            for strKey, strValue in strMode.items():
                url = '%sstrMode=%s' % ('http://isin.twse.com.tw/isin/C_public.jsp?', strKey)
                resp = requests.get(url, headers=twse_headers)
                resp.encoding ='dbcs'
                bs4soup = BeautifulSoup(resp.text, 'lxml')
                h4table = bs4soup.find("table", {"class" : "h4"})
                lastRow = None
                for row in h4table.find_all("tr"):
                    data = []
                    for col in row.find_all('td'):
                        each_col = col.text.strip().split('\u3000')
                        for eachcol in each_col:
                            data.append(eachcol)
                    if data and data[0] == '有價證券代號及名稱':
                        data[0] = '有價證券名稱'
                        data.insert(0, '有價證券代號')
                        data.append('證券類別')
                        if strKey == '2':
                            composedata.append(data)
                        continue
                    if len(data) == 1:
                        lastRow = data
                        if lastRow[0] in ('股票', '特別股', 'ETF', '臺灣存託憑證'):
                            lastRow[0] = '%s%s' % (strValue, lastRow[0])
                        continue
                    if lastRow:
                        composedata.append(data + lastRow)
                    else:
                        print('error!')
                        break
    except Exception as e:
        print('整理 %s 資料時發生錯誤: %s' % (accquiredfile, e))
        sys.exit(1)
    try:
        df = pd.DataFrame(composedata[1:], columns=composedata[0])
        df.rename(columns={
                '有價證券代號': 'symbol_id',
                '國際證券辨識號碼(ISIN Code)': 'isincode',
                '證券類別': 'market',
                }, inplace=True)
        df.drop(['有價證券名稱', '上市日', '市場別', '產業別', 'CFICode', '備註'], axis=1, inplace=True)
        df.sort_values('symbol_id', inplace=True)
        df.set_index('symbol_id', inplace=True)
    except Exception as e:
        print('整理 %s merge 資料時發生錯誤: %s' % (accquiredfile, e))
        sys.exit(1)
    return df.copy()

In [19]:
# 上市上櫃國際證券辨識號碼表
url, progress = 'http://isin.twse.com.tw/isin/C_public.jsp?', '上市上櫃國際證券辨識號碼表'
print('正在讀取 %s 請稍候...' % progress)
self_df = get_isincode(url, progress)
self_df

正在讀取 上市上櫃國際證券辨識號碼表 請稍候...


,isincode,market
symbol_id,,
0050,TW0000050004,上市ETF
0051,TW0000051002,上市ETF
0052,TW0000052000,上市ETF
0053,TW0000053008,上市ETF
0055,TW0000055003,上市ETF
...,...,...
9951,TW0009951004,上櫃股票
9955,TW0009955005,上市股票
9958,TW0009958009,上市股票


In [20]:
# 再次合併兩組資料
base_df = base_df.join(self_df)
base_df

,abb_zhname,name,registration,industry,tax_ID,found,listing,abb_enname,website,currency,par_value,description,type,enname,isincode,market
symbol_id,,,,,,,,,,,,,,,,
0050,元大台灣50,元大台灣卓越50證券投資信託基金,NaN,NaN,98764389,2003-06-25,2003-06-30,NaN,NaN,NaN,NaN,臺灣50指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Top 50 ETF,TW0000050004,上市ETF
0051,元大中型100,元大台灣中型100證券投資信託基金,NaN,NaN,48772770,2006-08-24,2006-08-31,NaN,NaN,NaN,NaN,臺灣中型100指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Mid-Cap 100 ETF,TW0000051002,上市ETF
0052,富邦科技,富邦台灣科技指數證券投資信託基金,NaN,NaN,48790067,2006-08-28,2006-09-12,NaN,NaN,NaN,NaN,臺灣資訊科技指數,國內成分證券指數股票型基金,Fubon Taiwan Technology Tracker Fund,TW0000052000,上市ETF
0053,元大電子,元大台灣ETF傘型證券投資信託基金之電子科技證券投資信託基金,NaN,NaN,48903196,2007-07-04,2007-07-16,NaN,NaN,NaN,NaN,電子類加權股價指數,國內成分證券指數股票型基金,Yuanta/P-shares Taiwan Electronics Tech ETF,TW0000053008,上市ETF
0055,元大MSCI金融,元大台灣ETF傘型證券投資信託基金之金融證券投資信託基金,NaN,NaN,48903175,2007-07-04,2007-07-16,NaN,NaN,NaN,NaN,MSCI台灣金融指數,國內成分證券指數股票型基金,Yuanta/P-shares MSCI Taiwan Financials ETF,TW0000055003,上市ETF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9949,琉園,琉園股份有限公司,,32.0,84767175,1994-04-07,2003-11-21,TITTOT,https://www.tittot.com,TWD,10.0,NaN,NaN,NaN,TW0009949008,上櫃股票
9950,萬國通,萬國通路股份有限公司,,3.0,73644351,1979-04-25,2004-02-17,EMINENT,https://www.eminent.com,TWD,10.0,NaN,NaN,NaN,TW0009950006,上櫃股票
9951,皇田,皇田工業股份有限公司,,5.0,73707377,1983-07-25,2003-12-18,MACAUTO,https://www.macauto-group.com/zh-hant/,TWD,10.0,NaN,NaN,NaN,TW0009951004,上櫃股票


In [22]:
import pickle

#with open('db_temp.pickle', 'b') as f:
#    base_df = pickle.load(f)

with open('security_table.pickle', 'wb') as f:
    pickle.dump(base_df, f)

base_df.to_csv(f'資料庫於 於 {datetime.strftime(todaytime, "%Y-%m-%d %H%M%S")} 匯入security 資料表的參考清單.csv', encoding='utf-8-sig')